# Домашние задание №2. Валидация и метрики

In [1]:
!pip install rectools

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy

from tqdm.auto import tqdm

from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset, Interactions
from rectools.models import RandomModel, PopularModel
from rectools.metrics.classification import Precision, Recall
from rectools.metrics.ranking import MRR, MAP, NDCG
from rectools.metrics.novelty import MeanInvUserFreq
from rectools.metrics.serendipity import Serendipity
from rectools.metrics import calc_metrics

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## Загрузка данных

In [3]:
data_dir_path = '/content/drive/MyDrive/recsys_course/data_kion'

data_interactions = pd.read_csv(f'{data_dir_path}/interactions.csv', parse_dates=['last_watch_dt'])

data_interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

print(data_interactions.shape)
data_interactions.head()

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [4]:
# оптимизируем датафрейм interactions для дальнейшей подачи в splitter
interactions = Interactions(data_interactions)
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [5]:
data_items = pd.read_csv(f'{data_dir_path}/items.csv')

print(data_items.shape)
data_items.head()

(15963, 14)


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [6]:
data_users = pd.read_csv(f'{data_dir_path}/users.csv')

print(data_users.shape)
data_users.head()

(840197, 5)


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


## Разбиение датасета для кросс-валидации

Сделаем 3 флода для кросс-валидации по неделе каждый с исключением холодных юзеров и айтемов и просмотренных айтемов.

In [7]:
splitter = TimeRangeSplitter("7D", 3)

In [8]:
splitter.get_test_fold_borders(interactions)

[(Timestamp('2021-08-02 00:00:00', freq='7D'),
  Timestamp('2021-08-09 00:00:00', freq='7D')),
 (Timestamp('2021-08-09 00:00:00', freq='7D'),
  Timestamp('2021-08-16 00:00:00', freq='7D')),
 (Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

## Обучение и валидация моделей

Протестируем следующие метрики:
1. 2 классификационные:

*   Precision
*   Recall

2. 2 ранжирующие + MAP:


*   MRR
*   NDCG
*   MAP


3. 2 beyond-accuracy:


*   Novelty
*   Serendipity

Все метрики считаем по порогам 1, 5, 10.

In [9]:
# возьмем две модели для тестирования различных метрик
models = {
    "random": RandomModel(random_state=32),
    "popular": PopularModel()
}

# будем расчитывать следующие метрик
metrics = {
    "precision@1": Precision(k=1),
    "precision@5": Precision(k=5),
    "precision@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "mrr@1": MRR(k=1),
    "mrr@5": MRR(k=5),
    "mrr@10": MRR(k=10),
    "ndcg@1": NDCG(k=1),
    "ndcg@5": NDCG(k=5),
    "ndcg@10": NDCG(k=10),
    "map@1": MAP(k=1),
    "map@5": MAP(k=5),
    "map@10": MAP(k=10),
    "novelty@1": MeanInvUserFreq(k=1),
    "novelty@5": MeanInvUserFreq(k=5),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@1": Serendipity(k=1),
    "serendipity@5": Serendipity(k=5),
    "serendipity@10": Serendipity(k=10)
}

K_RECOS = 10

In [10]:
# функция для кросс-валидации
# для каждого фолда разделяем датасет на train/test
# затем обучаем можели, выдаем рекомендации и считаем метрики
def cross_validate(models, metrics, interactions, splitter, k_recos):
  results = []

  fold_iterator = splitter.split(interactions, collect_fold_stats=True)

  for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=3):
      print(f"\n==================== Fold {fold_info['i_split']}")
      print(fold_info)

      df_train = interactions.df.iloc[train_ids]
      dataset = Dataset.construct(df_train)

      df_test = interactions.df.iloc[test_ids][Columns.UserItem]
      test_users = np.unique(df_test[Columns.User])

      catalog = df_train[Columns.Item].unique()

      for model_name, model in models.items():
          model = deepcopy(model)
          model.fit(dataset)
          recos = model.recommend(
              users=test_users,
              dataset=dataset,
              k=K_RECOS,
              filter_viewed=True,
          )
          metric_values = calc_metrics(
              metrics,
              reco=recos,
              interactions=df_test,
              prev_interactions=df_train,
              catalog=catalog,
          )
          res = {"fold": fold_info["i_split"], "model": model_name}
          res.update(metric_values)
          results.append(res)
  return results

In [11]:
%%time

cv_results = cross_validate(models, metrics, interactions, splitter, K_RECOS)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 4266013, 'train_users': 797423, 'train_items': 15237, 'test': 263681, 'test_users': 98184, 'test_items': 6602}

==================== Fold 1
{'i_split': 1, 'start': Timestamp('2021-08-09 00:00:00', freq='7D'), 'end': Timestamp('2021-08-16 00:00:00', freq='7D'), 'train': 4649162, 'train_users': 850489, 'train_items': 15415, 'test': 279422, 'test_users': 103511, 'test_items': 6698}

==================== Fold 2
{'i_split': 2, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 5051815, 'train_users': 906071, 'train_items': 15577, 'test': 298878, 'test_users': 110076, 'test_items': 6679}
CPU times: user 1min 58s, sys: 3.1 s, total: 2min 1s
Wall time: 2min 3s


In [12]:
# усредняем метрики по фолдам
pivot_results = pd.DataFrame(cv_results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean"])
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
    .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
)

,precision@1,recall@1,precision@5,recall@5,precision@10,recall@10,ndcg@1,ndcg@5,ndcg@10,mrr@1,mrr@5,mrr@10,map@1,map@5,map@10,novelty@1,novelty@5,novelty@10,serendipity@1,serendipity@5,serendipity@10
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
model,,,,,,,,,,,,,,,,,,,,,
random,0.000221,0.000072,0.000202,0.000365,0.000193,0.000693,0.000221,0.000208,0.000200,0.000221,0.000485,0.000604,0.000072,0.000169,0.000211,15.614137,15.612989,15.613009,0.000006,0.000007,0.000007
popular,0.076432,0.042720,0.052402,0.137413,0.033903,0.173492,0.076432,0.057932,0.043084,0.076432,0.131669,0.138603,0.042720,0.078295,0.084109,2.377055,3.066979,3.713390,0.000002,0.000003,0.000002


## Визуализация

In [13]:
# функция для визуализации просмотров юзеров и рекомендаций
def visualize(model, interactions_data, users, k_recos, item_data):
  dataset = Dataset.construct(interactions_data)
  recos = model.recommend(
       users=users,
       dataset=dataset,
       k=k_recos,
       filter_viewed=True,
  )
  for user_id in users:
    item_data_selected = item_data[['item_id', 'content_type', 'title',	'title_orig',	'release_year',	'genres']]
    item_data_selected.insert(len(item_data_selected.columns), 'num_of_views', interactions_data.groupby('item_id').agg({'user_id': 'count'}))
    user_viewed = interactions_data.loc[interactions_data['user_id'] == user_id].merge(item_data_selected, on="item_id")
    user_recos = recos.loc[recos['user_id'] == user_id].merge(item_data_selected, on="item_id")
    print(f'User {user_id} watched:\n')
    display(user_viewed)
    print(f'\nRecommendations for user {user_id}:\n')
    display(user_recos)
    print('\n====================\n')

In [14]:
model = PopularModel()
dataset = Dataset.construct(data_interactions)
model.fit(dataset)
USERS = [666262, 672861, 955527]

In [15]:
visualize(model, data_interactions, users=USERS, k_recos=K_RECOS, item_data=data_items)

User 666262 watched:



,user_id,item_id,datetime,weight,watched_pct,content_type,title,title_orig,release_year,genres,num_of_views
0,666262,7957,2021-05-12,2052.0,32.0,film,Последний викинг,The Lost Viking,2018.0,"боевики, историческое, приключения",1.0
1,666262,4785,2021-05-12,1946.0,28.0,film,Робин Гуд: Начало,Robin Hood,2018.0,"боевики, триллеры, приключения",1.0
2,666262,12981,2021-05-14,10292.0,100.0,film,Томирис,Tomiris,2020.0,"боевики, драмы, историческое, военные",5.0



Recommendations for user 666262:



,user_id,item_id,score,rank,content_type,title,title_orig,release_year,genres,num_of_views
0,666262,10440,202457.0,1,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",4.0
1,666262,15297,193123.0,2,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",20.0
2,666262,9728,132865.0,3,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры",143.0
3,666262,13865,122119.0,4,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",821.0
4,666262,4151,91167.0,5,series,Секреты семейной жизни,NaN,2021.0,комедии,10.0
5,666262,3734,74803.0,6,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,NaN
6,666262,2657,68581.0,7,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",725.0
7,666262,4880,55043.0,8,series,Афера,Afera,2021.0,комедии,5.0
8,666262,142,45367.0,9,film,Маша,Masha,2020.0,"драмы, триллеры",15.0
9,666262,6809,40372.0,10,film,Дуров,NaN,2021.0,документальное,14.0




User 672861 watched:



...

,user_id,item_id,score,rank,content_type,title,title_orig,release_year,genres,num_of_views
0,672861,10440,202457.0,1,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",4.0
1,672861,15297,193123.0,2,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",20.0
2,672861,9728,132865.0,3,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры",143.0
3,672861,13865,122119.0,4,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",821.0
4,672861,4151,91167.0,5,series,Секреты семейной жизни,NaN,2021.0,комедии,10.0
5,672861,3734,74803.0,6,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,NaN
6,672861,2657,68581.0,7,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",725.0
7,672861,4880,55043.0,8,series,Афера,Afera,2021.0,комедии,5.0
8,672861,142,45367.0,9,film,Маша,Masha,2020.0,"драмы, триллеры",15.0
9,672861,6809,40372.0,10,film,Дуров,NaN,2021.0,документальное,14.0




User 955527 watched:



,user_id,item_id,datetime,weight,watched_pct,content_type,title,title_orig,release_year,genres,num_of_views
0,955527,1183,2021-06-02,40.0,1.0,film,Стань легендой! Бигфут Младший,The Son of Bigfoot,2017.0,"мультфильм, фэнтези, приключения, комедии",1.0
1,955527,13371,2021-05-04,686.0,11.0,film,Пеле: Рождение легенды,Pele: Birth of a Legend(aka Pele),2016.0,"драмы, спорт, биография",2.0
2,955527,4725,2021-06-02,255.0,4.0,film,Лобановский навсегда,Lobanovskiy Forever,2016.0,"спорт, биография, документальное",7.0
3,955527,1238,2021-06-02,556.0,7.0,film,Диего Марадона,Diego Maradona,2019.0,"спорт, биография, документальное",642.0



Recommendations for user 955527:



,user_id,item_id,score,rank,content_type,title,title_orig,release_year,genres,num_of_views
0,955527,10440,202457.0,1,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",4.0
1,955527,15297,193123.0,2,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",20.0
2,955527,9728,132865.0,3,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры",143.0
3,955527,13865,122119.0,4,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",821.0
4,955527,4151,91167.0,5,series,Секреты семейной жизни,NaN,2021.0,комедии,10.0
5,955527,3734,74803.0,6,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,NaN
6,955527,2657,68581.0,7,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",725.0
7,955527,4880,55043.0,8,series,Афера,Afera,2021.0,комедии,5.0
8,955527,142,45367.0,9,film,Маша,Masha,2020.0,"драмы, триллеры",15.0
9,955527,6809,40372.0,10,film,Дуров,NaN,2021.0,документальное,14.0
